In [ ]:

# Generalized Code

# 1. Importing Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error
from sklearn.preprocessing import StandardScaler
from lightgbm import LGBMRegressor

import warnings
warnings.filterwarnings("ignore")

# 2. Loading Dataset Function
def load_dataset(file_path):
    return pd.read_excel(file_path)

# 3. Preprocessing Function
def preprocess_data(df, target_col, drop_columns=[]):
    # Dropping specified columns
    df = df.drop(columns=drop_columns, errors='ignore')
    
    # Ensure all columns are in lowercase
    df.columns = df.columns.str.lower()
    
    # If 'issuedate' is in the dataframe and is of type datetime
    if 'issuedate' in df.columns and np.issubdtype(df['issuedate'].dtype, np.datetime64):
        df["day"] = df["issuedate"].dt.dayofweek
        df["month"] = df["issuedate"].dt.month
        df["year"] = df["issuedate"].dt.year
        df = df.drop(columns='issuedate', errors='ignore')
    
    # Splitting dataset into train and test
    X = df.drop(columns=target_col, errors='ignore')
    y = df[target_col]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    return X_train, X_test, y_train, y_test

# 4. Model Training and Prediction Function
def train_and_predict(X_train, y_train):
    model = LGBMRegressor()
    model.fit(X_train, y_train)
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    return model, y_train_pred, y_test_pred

# 5. Evaluation Function
def evaluate_model(y_true, y_pred):
    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_true, y_pred)
    return mae, mse, rmse, r2

# 6. Main Execution
if __name__ == "__main__":
    # Load your dataset
    df = load_dataset('path_to_your_dataset.xlsx')
    
    # Preprocess your data
    X_train, X_test, y_train, y_test = preprocess_data(df, target_col='your_target_column', drop_columns=['columns_to_drop'])
    
    # Train and Predict
    model, y_train_pred, y_test_pred = train_and_predict(X_train, y_train)
    
    # Evaluate
    train_mae, train_mse, train_rmse, train_r2 = evaluate_model(y_train, y_train_pred)
    test_mae, test_mse, test_rmse, test_r2 = evaluate_model(y_test, y_test_pred)
    
    # Print Results
    print(f'Train set evaluation:\nMAE={train_mae}, MSE={train_mse}, RMSE={train_rmse}, R2={train_r2}')
    print(f'Test set evaluation:\nMAE={test_mae}, MSE={test_mse}, RMSE={test_rmse}, R2={test_r2}')
